In [1]:
from twython import Twython as tw
from twython import TwythonError

In [2]:
# App keys and secrets stored as environment variables
import os
APP_KEY = os.environ.get('TWITTER_APP_KEY')
APP_SECRET = os.environ.get('TWITTER_APP_SECRET')

Access_Token = os.environ.get('TWITTER_ACCESS_TOKEN')
Access_Secret = os.environ.get('TWITTER_ACCESS_SECRET')

twitter = tw(APP_KEY, APP_SECRET, Access_Token, Access_Secret)

In [10]:
import pandas as pd

In [6]:
# Define a number of queries

query_terms = {'bitcoin', 'altcoin', 'cryptocurrency', 'ethereum'}

In [7]:
# Then loop through queries
q_collection = {}

for x in query_terms:
    q_collection[x] = {'q': x,  
        'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        }

In [8]:
# Search those queries on Twitter and create a dictionary of responses
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_ID': []}  
for a in query_terms:
    for status in twitter.search(**q_collection[a])['statuses']:  
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['text'].append(status['text'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['user_ID'].append(status['user'])
        


In [11]:
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  

In [ ]:
# Store list for messing around with later
import time
timestr = time.strftime("%Y%m%d")
filename = 'tweets-'+timestr

df.to_csv(os.path.join(path,filename))

In [12]:
#Remove duplicates to find individual users

df = df.drop_duplicates(subset='user', keep='first', inplace=False)
#df.head(20)

In [13]:
# Pull user names & ID's and store

influencers = df.loc[:, 'user']

In [14]:
# Storing the newly found users in a Twitter list. 

my_user_id = 705469309
my_list_name = 'tweet_smart_list'
# Create dictionary of my lists
my_lists_dict = twitter.show_owned_lists(user_id = my_user_id)  

In [15]:
#Split so that running code doesn't keep sending requests to Twitter.

my_lists_df = pd.DataFrame(my_lists_dict['lists']) 
names = my_lists_df.loc[my_lists_df.name == my_list_name]
my_list_id = int(names.id)
list_slug = names.slug

# Stores users in twitter list, printing any errors for debugging
for user in influencers:
    try:
        twitter.add_list_member(list_id = my_list_id, screen_name=user)
    except TwythonError as e:
        print(e)